In [2]:
import json
import csv

# Specify the input and output file paths
input_file = 'NIKL_AU_2023_COMPETITION_v1.0/nikluge-au-2022-test.jsonl'
output_file = 'NIKL_AU_2023_COMPETITION_v1.0/nikluge-au-2022-test.csv'

data = []

# Read the JSONL file
with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        # Remove any leading/trailing whitespace
        line = line.strip()
        if not line:
            continue  # Skip empty lines
        # Parse the JSON object
        record = json.loads(line)
        data.append(record)

# Write data to a CSV file
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    # Define the CSV column headers
    fieldnames = ['id', 'input', 'output']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header row
    writer.writeheader()

    # Write each JSON object as a row in the CSV file
    for record in data:
        writer.writerow(record)
